# ClusDur Task 1.3 Notebook

## Results

![Iperf](results_isolated_net.png)

## Reproducing

- Set up nodes using either node_setup.py if you have access to our project, or by following the notebook set up instructions below.
- If you are using a stock image you need to manually install iperf3 on both nodes.
- On one compute node run user@cnode1> iperf3 -s, on the other run user@cnode2> iperf3 -c $CNODE1_IP_OR_HOSTNAME -J > results.json, then store the file.
- Replace the contents of results_closed_network.json in the artifact with your results and run the plotting cell at the bottom.

## Set up nodes from this notebook.

### Generic project set up

In [ ]:
import chi

# project set up
chi.set('project_name', 'CHI-221014')
chi.use_site('CHI@Purdue')

### Isolated Network

In [ ]:
import chi.network

net_name = "stbl"
subnet_name = "private-subnet-stbl-0"
rout_name = "public_router"

# automatically set up the network
stbl_net = chi.network.create_network(net_name)
subnet = chi.network.create_subnet(subnet_name, chi.network.get_network_id(net_name))
# may get the error: "IpAddressGenerationFailureClient: No more IP addresses available on network"
router = chi.network.create_router(rout_name, "public")
chi.network.add_subnet_to_router(chi.network.get_router_id(rout_name), chi.network.get_subnet_id(subnet_name))

### Leases

We attempt to provision the same hardware used to obtain our data, but some or all of it may not be available. If the leases by name don't work, run the next cell as a fallback.

In [ ]:
# attempt to lease the same head node.

import chi.lease

head_node = []
chi.lease.add_node_reservation(head_node, resource_properties=["==", "$node_name", "snyder-a003"], count=1)
head_lease = chi.lease.create_lease("CD_Head_Lease", reservations=head_node)
head_ready = chi.lease.wait_for_active(head_lease['id'])

In [ ]:
# lease any head node if above failed.

import chi.lease

head_node = []
chi.lease.add_node_reservation(head_node, node_type='indyscc_head', count=1)
head_lease = chi.lease.create_lease("CD_Head_Lease", reservations=head_node)
head_ready = chi.lease.wait_for_active(head_lease['id'])

In [ ]:
# compute node lease by node_name

import chi.lease

compute_nodes = []
chi.lease.add_node_reservation(compute_nodes, count=1, resource_properties=["==", "$node_name", "rice-a443"])
chi.lease.add_node_reservation(compute_nodes, count=1, resource_properties=["==", "$node_name", "rice-a447"])
compute_lease = chi.lease.create_lease("CD_Compute_Lease", reservations=compute_nodes)
compute_ready = chi.lease.wait_for_active(compute_lease['id'])

In [ ]:
# lease any compute nodes if above failed.

import chi.lease

compute_nodes = []
chi.lease.add_node_reservation(compute_nodes, count=2, node_type='indyscc_compute')
compute_lease = chi.lease.create_lease("CD_Compute_Lease", reservations=compute_nodes)
compute_ready = chi.lease.wait_for_active(compute_lease['id'])

### Servers

If you are running outside the ClusDur project you will not have our image and you may or may not have a keypair. If you do not have the 'indyscc_head-node' image change the image in this file to a stock image. You must also set the key to a previously generated keypair, or generate a new keypair via the GUI.

In [ ]:
import chi.lease, chi.server

# Project default image.
image = 'indyscc_head-node' # replace with a stock image if running out of project
key = 'CHANGE_ME' # replace with your generated keypair

head = chi.server.create_server(
    "CD_Head",
    reservation_id=chi.lease.get_node_reservation(head_lease['id']),
    image_name=image,
    network_name=net_name,
    key_name=key
)

compute = chi.server.create_server(
    "CD_Compute",
    reservation_id=chi.lease.get_node_reservation(compute_lease['id']),
    image_name=image,
    network_name=net_name,
    key_name=key
)


### Run nodes

We now actually start the nodes and allocate a floating IP to the head node so we can SSH in.

In [ ]:
import chi.server

hserv_start = chi.server.wait_for_active(head.id)
cserv_start = chi.server.wait_for_active(compute.id)
chi.server.associate_floating_ip(head.id)
print("Instances should now be running")

## Running iperf3

Unfortunately the iperf3 test between the compute nodes has to be done manually, assuming your ssh key is set up correctly you will have to ssh into the head node. Then log in to one compute node and run iperf -s, and run iperf3 -c $CNODE1_IP_OR_HOSTNAME -J > results.json on the other.

## Plotting

The following code should plot the results, if you are trying to reproduce, replace the contents of results_closed_network.json with your results from the last step.

In [ ]:
from matplotlib import pyplot as plt
import json


def read_data(file):
    data = []
    with open(file) as rfile:
        rjson = rfile.read()
        rdata = json.loads(rjson)
        intervals = rdata["intervals"]
        for interval in intervals:
            data.append(interval["sum"]["bits_per_second"])
    return data


def plot(data):
    plt.plot(data)
    plt.xlabel("interval number")
    plt.ylabel("bits per second")
    plt.show()


def main():
    data = read_data("results_closed_network.json")
    plot(data)


if __name__ == "__main__":
    main()